In [1]:
script_start_time = time()
println("\nimporting packages...")

using Pkg
using Statistics
using LinearAlgebra
# using Eirene
using StatsBase
using JLD
using Plots
Pkg.add("ColorSchemes")
using ColorSchemes
using JSON
using Pkg
Pkg.add("StatsPlots")
using StatsPlots

include("helper_functions.jl")
println("packages and functions imported")
printstyled("Elapsed time = $(time() - script_start_time) seconds \n \n", color = :yellow)

HTML{String}("<script>\n// Immediately-invoked-function-expression to avoid global variables.\n(function() {\n    var warning_div = document.getElementById(\"webio-warning-6061125794340041810\");\n    var hide = function () {\n        var script = document.getElementById(\"webio-setup-3521257105137419716\");\n        var parent = script && script.parentElement;\n        var grandparent = parent && parent.parentElement;\n        if (grandparent) {\n            grandparent.style.display = \"none\";\n        }\n        warning_div.style.display = \"none\";\n    };\n    if (typeof Jupyter !== \"undefined\") {\n        console.log(\"WebIO detected Jupyter notebook environment.\");\n        // Jupyter notebook.\n        var extensions = (\n            Jupyter\n            && Jupyter.notebook.config.data\n            && Jupyter.notebook.config.data.load_extensions\n        );\n        if (extensions && extensions[\"webio-jupyter-notebook\"]) {\n            // Extension already loaded.\n            console.log(\"Jupyter WebIO nbextension detected; not loading ad-hoc.\");\n            hide();\n            return;\n        }\n    } else if (window.location.pathname.includes(\"/lab\")) {\n        // Guessing JupyterLa\n        console.log(\"Jupyter Lab detected; make sure the @webio/jupyter-lab-provider labextension is installed.\");\n        hide();\n        return;\n    }\n})();\n\n</script>\n<p\n    id=\"webio-warning-6061125794340041810\"\n    class=\"output_text output_stderr\"\n    style=\"padding: 1em; font-weight: bold;\"\n>\n    Unable to load WebIO. Please make sure WebIO works for your Jupyter client.\n    For troubleshooting, please see <a href=\"https://juliagizmos.github.io/WebIO.jl/latest/providers/ijulia/\">\n    the WebIO/IJulia documentation</a>.\n    <!-- TODO: link to installation docs. -->\n</p>\n")


importing packages...
  Updating registry at `/opt/julia/registries/General`


┌ Warning: Some registries failed to update:
│     — `/opt/julia/registries/General` — registry dirty
└ @ Pkg.Types /buildworker/worker/package_linux64/build/usr/share/julia/stdlib/v1.2/Pkg/src/Types.jl:1171


 Resolving package versions...
  Updating `/opt/julia/environments/v1.2/Project.toml`
 [no changes]
  Updating `/opt/julia/environments/v1.2/Manifest.toml`
 [no changes]
 Resolving package versions...
  Updating `/opt/julia/environments/v1.2/Project.toml`
 [no changes]
  Updating `/opt/julia/environments/v1.2/Manifest.toml`
 [no changes]


┌ Info: Recompiling stale cache file /opt/julia/compiled/v1.2/StatsPlots/SiylL.ji for StatsPlots [f3b207a7-027a-5e70-b257-86293d7955fd]
└ @ Base loading.jl:1240


packages and functions imported
Elapsed time = 42.16188406944275 seconds 
 


In [2]:
### Set parameters

# Read from config file
config_file = "config090920.json"
config = read_config("$(homedir())/configs/$(config_file)")

# Parameters for all graphs
const NNODES = config["NNODES"]
const MAXDIM = config["MAXDIM"]
const NREPS = config["NREPS"]
const DATE_STRING = config["DATE_STRING"]


### Locate data and find nametags
read_dir = "$(homedir())/processed_data/results/$(NNODES)nodes"
betti_files = filter(x->occursin("bettiBars",x), readdir(read_dir))
betti_files = filter(x->occursin(DATE_STRING,x), betti_files)
println("Located the following graph files:")
for betti_file in betti_files
    println(betti_file)
end


# Locate the nametags
nametags = []
for betti_file in betti_files
    tag = split(split(betti_file, "$(DATE_STRING)_")[2], "_bettiBars")[1]
    nametags = [nametags; tag]
end

nametags = unique(nametags)

Located the following graph files:
IID_70nodes_20reps_090920_backward_bettiBars.jld
IID_70nodes_20reps_090920_forward_bettiBars.jld
IID_70nodes_20reps_090920_thresh03_edge725_noiseOnly_bettiBars.jld
IID_70nodes_20reps_090920_thresh03_edge725_threshold_bettiBars.jld
IID_70nodes_20reps_090920_thresh03_edge725_threshold_bettiBars_postnoise.jld
IID_70nodes_20reps_090920_thresh03_edge725_threshold_bettiBars_prenoise.jld
IID_70nodes_20reps_090920_thresh04_edge966_noiseOnly_bettiBars.jld
IID_70nodes_20reps_090920_thresh04_edge966_threshold_bettiBars.jld
IID_70nodes_20reps_090920_thresh04_edge966_threshold_bettiBars_postnoise.jld
IID_70nodes_20reps_090920_thresh04_edge966_threshold_bettiBars_prenoise.jld
IID_70nodes_20reps_090920_thresh05_edge1208_noiseOnly_bettiBars.jld
IID_70nodes_20reps_090920_thresh05_edge1208_threshold_bettiBars.jld
IID_70nodes_20reps_090920_thresh05_edge1208_threshold_bettiBars_postnoise.jld
IID_70nodes_20reps_090920_thresh05_edge1208_threshold_bettiBars_prenoise.jld
cos

8-element Array{Any,1}:
 "backward"                   
 "forward"                    
 "thresh03_edge725_noiseOnly" 
 "thresh03_edge725_threshold" 
 "thresh04_edge966_noiseOnly" 
 "thresh04_edge966_threshold" 
 "thresh05_edge1208_noiseOnly"
 "thresh05_edge1208_threshold"

In [3]:
for nametag in nametags
    
    ## Create one large plot with all of the betti curves
    betti_files_nametag = filter(x -> occursin("$(nametag)",x), betti_files)
    model_names = [split(betti_file_nametag,"_")[1] for betti_file_nametag in betti_files_nametag]
  

    ### Create one large plot with all of the betti curves
    nModels = length(betti_files_nametag)
    nEdges = binomial(NNODES, 2)
    bettiBarAll = zeros(NREPS, MAXDIM, nModels)
    muBarAll = zeros(NREPS, MAXDIM, nModels)
    nuBarAll = zeros(NREPS, MAXDIM, nModels)
    
    for (i,betti_file_nametag) in enumerate(betti_files_nametag)

        # Read in Betti curves
        bettiBar_dict = load("$(read_dir)/$(betti_file_nametag)")
        bettiBarAll[:,:, i] = bettiBar_dict["bettiBarArray"]
        muBarAll[:,:, i] = bettiBar_dict["muBarArray"]
        nuBarAll[:,:, i] = bettiBar_dict["nuBarArray"]

    end


    println("$(nametag) Size bettiBarAll is $(size(bettiBarAll))")



    # Plot
    p1a = violin(bettiBarAll[:,1,:],
        legend = false, alpha = 0.4, title = "dim 1")
    p1b = violin(bettiBarAll[:,2,:],
        legend = false, alpha = 0.4, title = "dim 2")
    p1c = violin(bettiBarAll[:,3,:],
        legend = false, alpha = 0.4, title = "dim 3")
    xlabel!("graphs")
    ylabel!("BettiBar")

    plot(p1a, p1b, p1c, layout = (3,1))

    savefig("../figures/all_bettiBars_$(nametag).pdf")


    p2a = violin(muBarAll[:,1,:],
        legend = false, alpha = 0.4, title = "dim 1")
    p2b = violin(muBarAll[:,2,:],
        legend = false, alpha = 0.4, title = "dim 2")
    p2c = violin(muBarAll[:,3,:],
        legend = false, alpha = 0.4, title = "dim 3")
    xlabel!("graphs")
    ylabel!("muBar")

    plot(p2a, p2b, p2c, layout = (3,1))

    savefig("../figures/all_muBars_$(nametag).pdf")


    p3a = violin(nuBarAll[:,1,:],
        legend = false, alpha = 0.4, title = "dim 1")
    p3b = violin(nuBarAll[:,2,:],
        legend = false, alpha = 0.4, title = "dim 2")
    p3c = violin(nuBarAll[:,3,:],
        legend = false, alpha = 0.4, title = "dim 3")
    xlabel!("graphs")
    ylabel!("muBar")

    plot(p3a, p3b, p3c, layout = (3,1))

    savefig("$(homedir())/figures/all_nuBars_$(nametag).pdf")

end

backward Size bettiBarAll is (20, 3, 6)
forward Size bettiBarAll is (20, 3, 7)
thresh03_edge725_noiseOnly Size bettiBarAll is (20, 3, 7)
thresh03_edge725_threshold Size bettiBarAll is (20, 3, 21)
thresh04_edge966_noiseOnly Size bettiBarAll is (20, 3, 7)
thresh04_edge966_threshold Size bettiBarAll is (20, 3, 21)
thresh05_edge1208_noiseOnly Size bettiBarAll is (20, 3, 7)
thresh05_edge1208_threshold Size bettiBarAll is (20, 3, 21)
